In [5]:
import tensorflow as tf
import os
import pickle
import numpy as np
import CifarData
CIFAR_DIR = "./../cifar-10-batches-py/"
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [6]:
# tensorboard
# 1. 指定面板图上显示的变量
# 2. 训练过程中将这些变量计算出来，输出到文件中
# 3. 文件解析 ./tensorboard --logdir=dir.

In [7]:


train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData.CifarData(train_filenames, True)
test_data = CifarData.CifarData(test_filenames, False)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [8]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
# [None], eg: [0,5,6,3]
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

# conv1: 神经元图， feature_map, 输出图像
conv1_1 = tf.layers.conv2d(x_image,
                           32, # output channel number
                           (3,3), # kernel size
                           padding = 'same',
                           activation = tf.nn.relu,
                           name = 'conv1_1')
conv1_2 = tf.layers.conv2d(conv1_1,
                           32, # output channel number
                           (3,3), # kernel size
                           padding = 'same',
                           activation = tf.nn.relu,
                           name = 'conv1_2')

# 16 * 16
pooling1 = tf.layers.max_pooling2d(conv1_2,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool1')


conv2_1 = tf.layers.conv2d(pooling1,
                           32, # output channel number
                           (3,3), # kernel size
                           padding = 'same',
                           activation = tf.nn.relu,
                           name = 'conv2_1')
conv2_2 = tf.layers.conv2d(conv2_1,
                           32, # output channel number
                           (3,3), # kernel size
                           padding = 'same',
                           activation = tf.nn.relu,
                           name = 'conv2_2')
# 8 * 8
pooling2 = tf.layers.max_pooling2d(conv2_2,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool2')

conv3_1 = tf.layers.conv2d(pooling2,
                           32, # output channel number
                           (3,3), # kernel size
                           padding = 'same',
                           activation = tf.nn.relu,
                           name = 'conv3_1')
conv3_2 = tf.layers.conv2d(conv3_1,
                           32, # output channel number
                           (3,3), # kernel size
                           padding = 'same',
                           activation = tf.nn.relu,
                           name = 'conv3_2')
# 4 * 4 * 32
pooling3 = tf.layers.max_pooling2d(conv3_2,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool3')
# [None, 4 * 4 * 32]
flatten = tf.layers.flatten(pooling3)
y_ = tf.layers.dense(flatten, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [10]:
def variable_summary(var, name):
    """Constructs summary for statistics of a variable"""
    with tf.name_scope(name):
        mean = tf.reduce_meanvar)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('mean', mean)
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.histogram('histogram', var)

# apply it to all nodes, 
with tf.name_scope('summary'):
    variable_summary(conv1_1, 'conv1_1')
    variable_summary(conv1_2, 'conv1_2')
    variable_summary(conv2_1, 'conv2_1')
    variable_summary(conv2_2, 'conv2_2')
    variable_summary(conv3_1, 'conv3_1')
    variable_summary(conv3_2, 'conv3_2')

"""
Create summary for tensorboard
"""
# we care about loss & accuracy the most:
#1) loss
loss_summary = tf.summary.scalar('loss', loss)
# 'loss': <10, 1.1>, <20, 1.08>
#2) accuracy
accuracy_summary = tf.summary.scalar('accuracy', accuracy)

#3) source_image
source_image = (x_image + 1) * 127.5  # reverse normalization
inputs_summary = tf.summary.image('inputs_image', source_image)

# two way to merge 'summary's
merged_summary = tf.summary.merge_all()
merged_summary_test = tf.summary.merge([loss_summary, accuracy_summary])


# create a paths(folders)
LOG_DIR = '.'
run_label = './run_vgg_tensorboard'
run_dir = os.path.join(LOG_DIR, run_label)
if not os.path.exists(run_dir):
    os.mkdir(run_dir)

# training folder 
train_log_dir = os.path.join(run_dir, 'train')
# prediction folder
test_log_dir = os.path.join(run_dir, 'test')
if not os.path.exists(train_log_dir):
    os.mkdir(train_log_dir)
if not os.path.exists(test_log_dir):
    os.mkdir(test_log_dir)

In [12]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

# output summary every 100 steps
output_summary_every_steps = 100


# train 10000: 70.45%
with tf.Session() as sess:
    sess.run(init)
    
    #
    train_writer = tf.summary.FileWriter(train_log_dir, sess.graph)
    test_writer = tf.summary.FileWriter(test_log_dir)
    
    fixed_test_batch_data, fixed_test_batch_labels = test_data.next_batch(batch_size)

    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        
        #
        eval_ops = [loss, accuracy, train_op]  # eval_op contains items to 'run'
        should_output_summary = ((i+1) % output_summary_every_steps == 0)
        if should_output_summary:
            eval_ops.append(merged_summary)
        eval_ops_results = sess.run(
                                    eval_ops,
                                    feed_dict={
                                        x: batch_data,
                                        y: batch_labels}
                                    )
        loss_val, acc_val = eval_ops_results[0:2]
        #
        if should_output_summary:
            train_summary_str = eval_ops_results[-1]
            train_writer.add_summary(train_summary_str, i+1)
            
            # in test, we choose the same batch to test!!
            test_summary_str = sess.run([merged_summary_test],
                                        feed_dict={
                                            x: fixed_test_batch_data,
                                            y: fixed_test_batch_labels,
                                        })[0]
            test_writer.add_summary(test_summary_str, i+1)
    
        if (i+1) % 100 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' 
                  % (i+1, loss_val, acc_val))
        if (i+1) % 1000 == 0:
            test_data = CifarData.CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                                        [accuracy],
                                        feed_dict = {
                                            x: test_batch_data, 
                                            y: test_batch_labels
                                        })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f' % (i+1, test_acc))
                
                
            

[Train] Step: 100, loss: 2.07766, acc: 0.30000
[Train] Step: 200, loss: 2.00673, acc: 0.25000
[Train] Step: 300, loss: 1.70037, acc: 0.40000
[Train] Step: 400, loss: 1.84755, acc: 0.30000
[Train] Step: 500, loss: 1.77263, acc: 0.25000
[Train] Step: 600, loss: 1.36166, acc: 0.65000
[Train] Step: 700, loss: 1.44902, acc: 0.65000
[Train] Step: 800, loss: 1.40021, acc: 0.45000
[Train] Step: 900, loss: 2.10793, acc: 0.30000
[Train] Step: 1000, loss: 1.13465, acc: 0.65000
(10000, 3072)
(10000,)
[Test ] Step: 1000, acc: 0.49950
[Train] Step: 1100, loss: 1.31510, acc: 0.55000
[Train] Step: 1200, loss: 1.22702, acc: 0.70000
[Train] Step: 1300, loss: 1.04106, acc: 0.65000
[Train] Step: 1400, loss: 1.45412, acc: 0.40000
[Train] Step: 1500, loss: 1.26331, acc: 0.65000
[Train] Step: 1600, loss: 1.22606, acc: 0.50000
[Train] Step: 1700, loss: 0.99629, acc: 0.65000
[Train] Step: 1800, loss: 1.08484, acc: 0.50000
[Train] Step: 1900, loss: 0.68985, acc: 0.70000
[Train] Step: 2000, loss: 1.00557, acc: 0

In [22]:
# go to log directory
# use this command to open tensorboad:
# $tensorboard --logdir=train:'train',test:'test'
# test: the folder under the log, containing events for test; 'test' is its name
# train: the folder under the log, containing events for train; 'train' is its name